In [41]:
import random
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings(action='ignore')


#머신러닝 모델링
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from scipy.stats import uniform
import numpy as np
import xgboost
from category_encoders import TargetEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [42]:
# wav 파일의 MFCC Feature와 상태정보를 합친 학습데이터를 불러옵니다.
train_df = pd.read_csv("open/train_mfcc_data.csv")

# 학습데이터를 모델의 input으로 들어갈 x와 label로 사용할 y로 분할
train_x = train_df.drop(columns=['id', 'covid19'])
train_y = train_df['covid19']


In [43]:
def onehot_encoding(ohe, x):
  # 학습데이터로부터 fit된 one-hot encoder (ohe)를 받아 transform 시켜주는 함수
  encoded = ohe.transform(x['gender'].values.reshape(-1,1))
  encoded_df = pd.DataFrame(encoded, columns=ohe.categories_[0])
  x = pd.concat([x.drop(columns=['gender']), encoded_df], axis=1)
  return x

In [44]:
ohe = OneHotEncoder(sparse=False)
ohe.fit(train_x['gender'].values.reshape(-1,1))
train_x = onehot_encoding(ohe,train_x)
train_x = train_x.drop(columns=['other'])

In [45]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=10)
print(train_x.shape)
print(train_y.shape)

(3044, 37)
(3044,)


In [46]:
pipe = make_pipeline(TargetEncoder(), SimpleImputer(), RandomForestClassifier())
dists = {
         'randomforestclassifier__max_depth':range(5,10), 'randomforestclassifier__n_estimators':range(100,200,10), 'randomforestclassifier__max_features':uniform(0,1)}
clf = RandomizedSearchCV(pipe, param_distributions=dists, n_iter=50, cv=5, scoring="recall", verbose=1, n_jobs=-1)

clf.fit(train_x, train_y)


Fitting 5 folds for each of 50 candidates, totalling 250 fits


/Users/jinyoochan/miniforge3/envs/cp1/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  elif pd.api.types.is_categorical(cols):
/Users/jinyoochan/miniforge3/envs/cp1/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  elif pd.api.types.is_categorical(cols):
/Users/jinyoochan/miniforge3/envs/cp1/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  elif pd.api.types.is_categorical(cols):
/Users/jinyoochan/miniforge3/envs/cp1/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  elif pd.

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('targetencoder', TargetEncoder()),
                                             ('simpleimputer', SimpleImputer()),
                                             ('randomforestclassifier',
                                              RandomForestClassifier())]),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'randomforestclassifier__max_depth': range(5, 10),
                                        'randomforestclassifier__max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x16c7bfc70>,
                                        'randomforestclassifier__n_estimators': range(100, 200, 10)},
                   scoring='recall', verbose=1)

In [47]:
# val데이터 Model 추론
preds = clf.predict(val_x)
# f1_score
print(f1_score(val_y, preds))

0.10344827586206896


In [48]:
# 위의 학습데이터를 전처리한 과정과 동일하게 test data에도 적용
test_x = pd.read_csv('open/test_mfcc_data.csv')
test_x = test_x.drop(columns=['id'])
# Data Leakage에 유의하여 train data로만 학습된 ohe를 사용
test_x = onehot_encoding(ohe, test_x)
test_x = test_x.drop(columns=["other"])

# Model 추론
preds = clf.predict(test_x)

In [49]:

submission = pd.read_csv('open/sample_submission.csv')
submission['covid19'] = preds
submission.to_csv('open/submit2.csv', index=False)